In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from datasets import load_dataset

import time
import nltk

nltk.download('wordnet')

/Users/jinhyunpark/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jinhyunpark/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Define Models and Tokenizer

In [2]:
# T5
tokenizer_t5_base = T5Tokenizer.from_pretrained("google-t5/t5-base")
model_t5_base = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base")

# BART 
tokenizer_bart_base = BartTokenizer.from_pretrained("facebook/bart-base")
model_bart_base = BartForConditionalGeneration.from_pretrained("facebook/bart-base")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


: 

In [ ]:
input_text_t5 = "translate English to French: I like to eat pizza"
t5_input_ids = tokenizer_t5_base.encode(input_text_t5, return_tensors='pt')
t5_outputs = model_t5_base.generate(t5_input_ids, max_new_tokens=999)
t5_output_text = tokenizer_t5_base.decode(t5_outputs[0], skip_special_tokens=True)
print(t5_output_text)